In [1]:
import sys
import numpy as np
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
# from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from random import random
import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [0]:
def get_Word(fp):
    word = []
    raw_text = open(fp).read().lower()
    raw_text = raw_text.split("\n")
    for row in raw_text:
        temp = row.split(" ")
        for i in range(len(temp)):
            word.append(temp[i])
    word.append("\n")
    print("Length of total word_list:-", len(word))
    return(word)

#First create a set of all of the distinct words,
# then creating a map of each word to a unique integer.
def get_Word_dict(word_list):
    uniq_word = sorted(list(set(word_list)))
    uniqword_to_int = dict((c, i) for i, c in enumerate(uniq_word))
    print("Total no. Words:- ", len(word_list))
    print("Total no. of unique words:- ", len(uniq_word))
    return(uniqword_to_int)

def get_int_to_word(word_list):
    uniq_word = sorted(list(set(word_list)))
    int_to_word = dict((i, c) for i, c in enumerate(uniq_word))
    return(int_to_word)

# prepare the dataset of input to output pairs encoded as integers
def Data_Preprocessing(fp, seq_no, word_dict):
    datax = []
    datay = []
    
    raw_text = open(fp).read().lower()
    raw_text = raw_text.split("\n")
    fdatax = open('data_input.csv', "w")
    for row in raw_text:
        temp = (row +" \n").split(" ")
        for i in range(len(temp)-seq_no):
            temp_word_int = []
            temp_word = []
            for index in range(0, seq_no):
                temp_word.append(temp[i+index])
                temp_word_int.append(word_dict[temp[i+index]])
#                print(temp_word_int)
            datax.append([temp_word_int])
            print(temp_word, ",", temp_word_int, file = fdatax)
            datay.append([word_dict[temp[i+seq_no]]])
    return(datax, datay)

def get_initial_last_Word(fp, seq_no):
    initial_word = []
    last_word = []

    raw_text = open(fp).readlines()
    for row in raw_text:
        temp= row.split(" ")
        temp_word = ""
        for index in range(0, seq_no):
            temp_word += temp[0+index]+" "
        initial_word.append(temp_word.rstrip(" "))
        last_word.append(temp[-1])
    
    dami = list(set(initial_word))
    dx = [x.split(" ") for x in dami]
    return(dx, list(set(last_word)))

In [0]:
def createModel(X,y):
    model = Sequential()
    model.add(LSTM(128, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(128))
    model.add(Dropout(0.2))
    model.add(Dense(y.shape[1], activation='softmax'))
    model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy', 'binary_crossentropy'])
    model.summary()
    return model

In [0]:
def plot_history(histories, key='binary_crossentropy'):
  plt.figure(figsize=(16,10))

  for name, history in histories:
    val = plt.plot(history.epoch, history.history['val_'+key],'--', label=name.title()+' Val')
    plt.plot(history.epoch, history.history[key], color=val[0].get_color(),label=name.title()+' Train')

  plt.xlabel('Epochs')
  plt.ylabel(key.replace('_',' ').title())
  plt.legend()

  plt.xlim([0,max(history.epoch)])

In [5]:
filename = 'class_3_p.csv'
seq = 3
fileN=open("generatedText.txt","w")
total_words = get_Word(filename)

Length of total word_list:- 30874


In [6]:
word_dict = get_Word_dict(total_words)
int_to_word = get_int_to_word(total_words)
initial_word, last_word = get_initial_last_Word(filename, seq)
#print(initial_word)
datax, datay = Data_Preprocessing(filename, seq, word_dict)
#print(datax, file=fileN)
no_patterns = len(datax)

Total no. Words:-  30874
Total no. of unique words:-  1011


In [0]:
X = np.reshape(datax, (no_patterns, seq, 1))
X = X / float(len(set(total_words)))
y = np_utils.to_categorical(datay)

In [9]:
y.shape

(27600, 1011)

In [15]:
model=None
model = createModel(X, y)
histories=model.fit(X, y, epochs=4, batch_size=128)
plot_history([('baseline', histories)])

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 3, 128)            66560     
_________________________________________________________________
dropout_5 (Dropout)          (None, 3, 128)            0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1011)              130419    
Total params: 328,563
Trainable params: 328,563
Non-trainable params: 0
_________________________________________________________________
Epoch 1/4
27600/27600 [==============================] - 11s 397us/step - loss: 0.0061 - acc: 0.9990 - binary_cros

KeyError: ignored

<Figure size 1152x720 with 0 Axes>

In [0]:
start = np.random.randint(0, len(initial_word)-1)
seed_text = initial_word[start]
print(seed_text)
seed = [word_dict[word] for word in seed_text]
print(seed)
generatedTextList= seed_text
flag = True
print(generatedTextList)
count=0

while (flag):
    x = np.reshape(seed, (1, len(seed), 1))
    x = x / float(len(set(total_words)))
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_word[index]
    generatedTextList.append(result)
    seq_in = [int_to_word[value] for value in seed]
#         sys.stdout.write(result+ " ")
#         print(result+ " ",end="")
    seed.append(index)
    seed = seed[1:len(seed)]
    if result == "\n" or count==10:
        flag = False
    count+=1

print(generatedTextList, file = fileN)

In [16]:
import numpy as np
from sklearn.model_selection import KFold

X = np.array([[1, 2], [3, 4], [1, 2], [3, 4]])
y = np.array([1, 2, 3, 4])
kf = KFold(n_splits=2)
kf.get_n_splits(X)

2